# Entry 6 notebook - EDA correlation notebook

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn')

### Load data

In [2]:
planets = pd.read_excel('../data/planets_moons.xlsx', index_col=0)
planets

,type,mass_1024kg,diameter_km,density_kg_m3,gravity_m_s2,escape_vel_km_s,rotation_period_hr,day_len_hr,distance_from_sun_106_km,perihelion_106 km,...,mean_temp_c,surface_pressure_bars,nbr_moons,rings,magnetic_field,equatorial_radius_km,mean_radius_km,"V(1,0) (mag)",geometric_albedo,atmospheric_mass_kg
name,,,,,,,,,,,,,,,,,,,,,
Mercury,planet,0.3300,4879.0,5427,3.7,4.3,1407.6,4222.6,57.9,46.0,...,167,1.000000e-14,0,No,Yes,2440.5300,2439.4000,-0.60,0.106,1.000000e+03
Venus,planet,4.8700,12104.0,5243,8.9,10.4,-5832.5,2802.0,108.2,107.5,...,464,9.200000e+01,0,No,No,6051.8000,6051.8000,-4.47,0.650,4.800000e+20
Earth,planet,5.9700,12756.0,5514,9.8,11.2,23.9,24.0,149.6,147.1,...,15,1.014000e+00,1,No,Yes,6378.1366,6371.0084,-3.86,0.367,1.400000e+21
Moon,moon,0.0730,3475.0,3340,1.6,2.4,655.7,708.7,149.6,147.1,...,-20,3.000000e-15,0,No,No,1737.5000,1737.4000,-0.08,0.120,1.000000e+05
Mars,planet,0.6420,6792.0,3933,3.7,5.0,24.6,24.7,227.9,206.6,...,-65,1.000000e-02,2,No,No,3396.1900,3389.5000,-1.52,0.150,2.500000e+16
Jupiter,planet,1898.0000,142984.0,1326,23.1,59.5,9.9,9.9,778.6,740.5,...,-110,2.000000e+00,79,Yes,Yes,71492.0000,69911.0000,-9.40,0.520,1.900000e+27
Saturn,planet,568.0000,120536.0,687,9.0,35.5,10.7,10.7,1433.5,1352.6,...,-140,1.000000e+03,82,Yes,Yes,60268.0000,58232.0000,-8.88,0.470,5.400000e+26
Titan,moon,0.1260,5149.4,1882,1.4,2.6,382.0,382.0,1433.5,1352.6,...,-179,1.600000e+00,0,No,No,2574.7000,2574.7000,-8.10,0.210,9.100000e+18
Uranus,planet,86.8000,51118.0,1271,8.7,21.3,-17.2,17.2,2872.5,2741.3,...,-195,1.000000e+03,27,Yes,Yes,25559.0000,25362.0000,-7.19,0.510,8.600000e+25


### Locate categorical variables

In [4]:
planets.select_dtypes('object')

,type,rings,magnetic_field
name,,,
Mercury,planet,No,Yes
Venus,planet,No,No
Earth,planet,No,Yes
Moon,moon,No,No
Mars,planet,No,No
Jupiter,planet,Yes,Yes
Saturn,planet,Yes,Yes
Titan,moon,No,No
Uranus,planet,Yes,Yes


### Convert categoricals

In [5]:
planets[['type', 'rings', 'magnetic_field']] = planets[['type', 'rings', 'magnetic_field']].astype('category')

In [6]:
for cat in planets.select_dtypes('category').columns.tolist():
    planets[cat] = planets[cat].cat.codes

In [7]:
planets[['type', 'magnetic_field', 'rings']].head()

,type,magnetic_field,rings
name,,,
Mercury,2,1,0
Venus,2,0,0
Earth,2,1,0
Moon,1,0,0
Mars,2,0,0


### Run correlations

In [26]:
pearson_feature_corr = pd.DataFrame(planets.corrwith(planets['atmospheric_mass_kg'], axis=0).reset_index()).rename(columns={0:'pearson_cor'})

In [27]:
spearman_feature_corr = pd.DataFrame(planets.corrwith(planets['atmospheric_mass_kg'], axis=0, method='spearman').reset_index()).rename(columns={0:'spearman_cor'})

### Combine and sort

Combined the pearson and spearman results to compare them.

Created a column with the absolute value of the spearman result. This allowed me to sort by the absolute value of the Spearman value. I don't care if it's a positive or negative relationship, just how strong the relationship is.

In [11]:
feature_corr = pearson_feature_corr.merge(spearman_feature_corr, on='index', how='inner')
feature_corr['abs_spearman'] = feature_corr['spearman_cor'].abs()
feature_corr.sort_values('abs_spearman', ascending=False)

,index,pearson_cor,spearman_cor,abs_spearman
25,atmospheric_mass_kg,1.000000,1.000000,1.000000
2,diameter_km,0.862912,0.945455,0.945455
22,mean_radius_km,0.864138,0.945455,0.945455
21,equatorial_radius_km,0.862915,0.945455,0.945455
5,escape_vel_km_s,0.910232,0.918182,0.918182
1,mass_1024kg,0.999894,0.918182,0.918182
7,day_len_hr,-0.242968,-0.881818,0.881818
23,"V(1,0) (mag)",-0.570149,-0.881818,0.881818
17,surface_pressure_bars,-0.003563,0.853247,0.853247
19,rings,0.577273,0.836660,0.836660


### Correlation method compairson

The different quantative values for correlation are interesting.

Spearman ranked diameter_km, mean_radius_km, and equatorial_radius_km all equally correlated to atmospheric_mass_kg, which would be expected as they all measure the same thing. Spearman also gave them more importance than Pearson.

Pearson ranked mass_1024kg more important than Spearman did, which would be good for my theory of chaning the mass will all the planet to hold onto more atmosphere. But having discussed the issue with a friend, I'm wondering if atmospheric mass is included in planetary mass. If this were a real world problem where the result mattered, I'd look into it. However, since this is a spur for a fictional story, I'm going to ignore it.

Spearman ranked escape_vel_km_s the same as mass_1024kg, and put both at about the same importance as diameter/radius. I would have expected gravity_m_s2 and density to be in with this feature set as mass, density, diameter, and gravity are [all mathematically related](https://earthobservatory.nasa.gov/features/GRACE/page2.php) and escape velocity is a direct result of gravity.

Based on the prevalence that magnetic field holds in the literature, I expected magnetic field to have a stronger correlation.

Several of the lower correlations have opposite relationships depending on which method is used. For example, orbital_period_days and distance_from_sun_106_km.

## Narrow features

Sabber and I originally decided 20 features was a good starting place. After the results of the visualization exercises in Entry 5 (where I ended up with 22 visualizations for 5-8 features), I decided to narrow this a little further. My cut off is now an absolute Spearman value greater than or equal to 0.5.

I also ordered the columns by most important to least important features (as related to atmospheric_mass_kg).

In [25]:
features = feature_corr[feature_corr['abs_spearman'] >= 0.5].sort_values('abs_spearman', ascending=False)['index'].tolist()
planets[features]

,atmospheric_mass_kg,diameter_km,mean_radius_km,equatorial_radius_km,mass_1024kg,escape_vel_km_s,"V(1,0) (mag)",day_len_hr,surface_pressure_bars,rings,gravity_m_s2,geometric_albedo,nbr_moons,density_kg_m3,magnetic_field,orbital_inclination_degrees,type,rotation_period_hr
name,,,,,,,,,,,,,,,,,,
Mercury,1.000000e+03,4879.0,2439.4000,2440.5300,0.3300,4.3,-0.60,4222.6,1.000000e-14,0,3.7,0.106,0,5427,1,7.0,2,1407.6
Venus,4.800000e+20,12104.0,6051.8000,6051.8000,4.8700,10.4,-4.47,2802.0,9.200000e+01,0,8.9,0.650,0,5243,0,3.4,2,-5832.5
Earth,1.400000e+21,12756.0,6371.0084,6378.1366,5.9700,11.2,-3.86,24.0,1.014000e+00,0,9.8,0.367,1,5514,1,0.0,2,23.9
Moon,1.000000e+05,3475.0,1737.4000,1737.5000,0.0730,2.4,-0.08,708.7,3.000000e-15,0,1.6,0.120,0,3340,0,5.1,1,655.7
Mars,2.500000e+16,6792.0,3389.5000,3396.1900,0.6420,5.0,-1.52,24.7,1.000000e-02,0,3.7,0.150,2,3933,0,1.9,2,24.6
Jupiter,1.900000e+27,142984.0,69911.0000,71492.0000,1898.0000,59.5,-9.40,9.9,2.000000e+00,1,23.1,0.520,79,1326,1,1.3,2,9.9
Saturn,5.400000e+26,120536.0,58232.0000,60268.0000,568.0000,35.5,-8.88,10.7,1.000000e+03,1,9.0,0.470,82,687,1,2.5,2,10.7
Titan,9.100000e+18,5149.4,2574.7000,2574.7000,0.1260,2.6,-8.10,382.0,1.600000e+00,0,1.4,0.210,0,1882,0,0.3,1,382.0
Uranus,8.600000e+25,51118.0,25362.0000,25559.0000,86.8000,21.3,-7.19,17.2,1.000000e+03,1,8.7,0.510,27,1271,1,0.8,2,-17.2
